# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 28 2022 1:08PM,242999,10,8334929,Eywa Pharma Inc.,Released
1,Jul 28 2022 1:00PM,242994,20,8294807-BO,"ACI Healthcare USA, Inc.",Released
2,Jul 28 2022 12:57PM,242997,16,8334328,Sartorius Bioprocess Solutions,Released
3,Jul 28 2022 12:57PM,242997,16,8334371,Sartorius Bioprocess Solutions,Released
4,Jul 28 2022 12:55PM,242996,10,Enova-10381,Emerginnova,Released
5,Jul 28 2022 12:55PM,242996,10,Enova-10382,Emerginnova,Released
6,Jul 28 2022 12:50PM,242911,19,GP07292022,"Granules Pharmaceuticals, Inc.",Executing
7,Jul 28 2022 12:43PM,242992,10,MTR-22-0240,"Senseonics, Incorporated",Released
8,Jul 28 2022 12:40PM,242991,10,0085914509,ISDIN Corporation,Released
9,Jul 28 2022 12:40PM,242991,10,0085914272,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
40,242992,Released,1
41,242994,Released,1
42,242996,Released,2
43,242997,Released,2
44,242999,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242992,NaN,NaN,1.0
242994,NaN,NaN,1.0
242996,NaN,NaN,2.0
242997,NaN,NaN,2.0
242999,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242886,7.0,0.0,13.0
242889,0.0,2.0,1.0
242898,9.0,6.0,1.0
242900,0.0,0.0,1.0
242901,1.0,0.0,5.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242886,7,0,13
242889,0,2,1
242898,9,6,1
242900,0,0,1
242901,1,0,5


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242886,7,0,13
1,242889,0,2,1
2,242898,9,6,1
3,242900,0,0,1
4,242901,1,0,5


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242886,7,,13
1,242889,,2,1
2,242898,9,6,1
3,242900,,,1
4,242901,1,,5


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 28 2022 1:08PM,242999,10,Eywa Pharma Inc.
1,Jul 28 2022 1:00PM,242994,20,"ACI Healthcare USA, Inc."
2,Jul 28 2022 12:57PM,242997,16,Sartorius Bioprocess Solutions
4,Jul 28 2022 12:55PM,242996,10,Emerginnova
6,Jul 28 2022 12:50PM,242911,19,"Granules Pharmaceuticals, Inc."
7,Jul 28 2022 12:43PM,242992,10,"Senseonics, Incorporated"
8,Jul 28 2022 12:40PM,242991,10,ISDIN Corporation
14,Jul 28 2022 12:39PM,242989,10,ISDIN Corporation
51,Jul 28 2022 12:39PM,242990,21,"NBTY Global, Inc."
52,Jul 28 2022 12:06PM,242986,20,"Exact-Rx, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 28 2022 1:08PM,242999,10,Eywa Pharma Inc.,,,1
1,Jul 28 2022 1:00PM,242994,20,"ACI Healthcare USA, Inc.",,,1
2,Jul 28 2022 12:57PM,242997,16,Sartorius Bioprocess Solutions,,,2
3,Jul 28 2022 12:55PM,242996,10,Emerginnova,,,2
4,Jul 28 2022 12:50PM,242911,19,"Granules Pharmaceuticals, Inc.",,1,
5,Jul 28 2022 12:43PM,242992,10,"Senseonics, Incorporated",,,1
6,Jul 28 2022 12:40PM,242991,10,ISDIN Corporation,,,6
7,Jul 28 2022 12:39PM,242989,10,ISDIN Corporation,,,37
8,Jul 28 2022 12:39PM,242990,21,"NBTY Global, Inc.",,,1
9,Jul 28 2022 12:06PM,242986,20,"Exact-Rx, Inc.",,4,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 28 2022 1:08PM,242999,10,Eywa Pharma Inc.,1,,
1,Jul 28 2022 1:00PM,242994,20,"ACI Healthcare USA, Inc.",1,,
2,Jul 28 2022 12:57PM,242997,16,Sartorius Bioprocess Solutions,2,,
3,Jul 28 2022 12:55PM,242996,10,Emerginnova,2,,
4,Jul 28 2022 12:50PM,242911,19,"Granules Pharmaceuticals, Inc.",,1,
5,Jul 28 2022 12:43PM,242992,10,"Senseonics, Incorporated",1,,
6,Jul 28 2022 12:40PM,242991,10,ISDIN Corporation,6,,
7,Jul 28 2022 12:39PM,242989,10,ISDIN Corporation,37,,
8,Jul 28 2022 12:39PM,242990,21,"NBTY Global, Inc.",1,,
9,Jul 28 2022 12:06PM,242986,20,"Exact-Rx, Inc.",1,4,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 28 2022 1:08PM,242999,10,Eywa Pharma Inc.,1,,
1,Jul 28 2022 1:00PM,242994,20,"ACI Healthcare USA, Inc.",1,,
2,Jul 28 2022 12:57PM,242997,16,Sartorius Bioprocess Solutions,2,,
3,Jul 28 2022 12:55PM,242996,10,Emerginnova,2,,
4,Jul 28 2022 12:50PM,242911,19,"Granules Pharmaceuticals, Inc.",,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 28 2022 1:08PM,242999,10,Eywa Pharma Inc.,1.0,NaN,NaN
1,Jul 28 2022 1:00PM,242994,20,"ACI Healthcare USA, Inc.",1.0,NaN,NaN
2,Jul 28 2022 12:57PM,242997,16,Sartorius Bioprocess Solutions,2.0,NaN,NaN
3,Jul 28 2022 12:55PM,242996,10,Emerginnova,2.0,NaN,NaN
4,Jul 28 2022 12:50PM,242911,19,"Granules Pharmaceuticals, Inc.",NaN,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 28 2022 1:08PM,242999,10,Eywa Pharma Inc.,1.0,0.0,0.0
1,Jul 28 2022 1:00PM,242994,20,"ACI Healthcare USA, Inc.",1.0,0.0,0.0
2,Jul 28 2022 12:57PM,242997,16,Sartorius Bioprocess Solutions,2.0,0.0,0.0
3,Jul 28 2022 12:55PM,242996,10,Emerginnova,2.0,0.0,0.0
4,Jul 28 2022 12:50PM,242911,19,"Granules Pharmaceuticals, Inc.",0.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2672631,73.0,9.0,9.0
15,485945,5.0,20.0,0.0
16,971836,5.0,0.0,0.0
19,1214545,14.0,25.0,24.0
20,1457620,52.0,9.0,8.0
21,242990,1.0,0.0,0.0
22,728913,1.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2672631,73.0,9.0,9.0
1,15,485945,5.0,20.0,0.0
2,16,971836,5.0,0.0,0.0
3,19,1214545,14.0,25.0,24.0
4,20,1457620,52.0,9.0,8.0
5,21,242990,1.0,0.0,0.0
6,22,728913,1.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,73.0,9.0,9.0
1,15,5.0,20.0,0.0
2,16,5.0,0.0,0.0
3,19,14.0,25.0,24.0
4,20,52.0,9.0,8.0
5,21,1.0,0.0,0.0
6,22,1.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,73.0
1,15,Released,5.0
2,16,Released,5.0
3,19,Released,14.0
4,20,Released,52.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21,22
Status,,,,,,,
Completed,9.0,0.0,0.0,24.0,8.0,0.0,0.0
Executing,9.0,20.0,0.0,25.0,9.0,0.0,2.0
Released,73.0,5.0,5.0,14.0,52.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21,22
0,Completed,9.0,0.0,0.0,24.0,8.0,0.0,0.0
1,Executing,9.0,20.0,0.0,25.0,9.0,0.0,2.0
2,Released,73.0,5.0,5.0,14.0,52.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21,22
0,Completed,9.0,0.0,0.0,24.0,8.0,0.0,0.0
1,Executing,9.0,20.0,0.0,25.0,9.0,0.0,2.0
2,Released,73.0,5.0,5.0,14.0,52.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()